# CIS(D)

In [1]:
import numpy as np
import pyci
import psi4
from threadpoolctl import threadpool_limits
from time import perf_counter


In [3]:
from pyci.configint.rcisd import CISD
basis = 'aug-cc-pvqz'
molfile = '../.xyz/Be.xyz'
ncore = 11
with threadpool_limits(limits=ncore, user_api='blas'):
    mol = pyci.utils.molecule(basis, molfile, wd='./', ncore=ncore, psi4mem='210 Gb',
                              numpymem=210, custom_basis=False, 
                              store_wfn=False, properties=['dipoles'], psi4options={'puream': False})


print('Begining CISD calculation .....\n')
start = perf_counter()
with threadpool_limits(limits=ncore, user_api='blas'):
    cisd = CISD(mol, options={'doubles': False}, ncore=ncore)
    ndim = sum(cisd.num_csfs)
    print(cisd.num_csfs, ndim)
    cisd.save_hcisd()
stop = perf_counter()


COMPUTING AO ELECTRON REPULSION INTEGRALS

Size of the ERI tensor will be 0.97 Gb

COMPUTING AO ONE ELECTRON INTEGRALS

Ground state SCF Energy : -14.57297602 

Nuclear repulsion energy : 0.00000000 

Total electronic energy : -14.57297602 

 ORBITAL INFORMATION  
-----------------------
Basis functions   : 105  
Molecular orbitals: 105  
Spin Orbitals     : 210  
Alpha Orbitals    : 2  
Beta Orbitals     : 2  
Occupied Orbitals : 2  
Virtual Orbitals  : 103  

Begining CISD calculation .....

[1, 206, 0, 0, 0, 0, 0] 207
Computing CISD Hamiltonian matrix with 11 cores 



In [4]:
HCIS = np.load('cimat.npz')['HCISD']

In [5]:
HCIS0 = HCIS - np.eye(sum(cisd.num_csfs))*mol.scf_energy
ECIS, CCIS = np.linalg.eigh(HCIS0)

In [11]:
ao_erints = mol.get_ao_erints()
mo_so_erints = mol.get_mo_so_eri(mol.mo_eps, mol.mo_coeff, ao_erints)
mo_so_erints = mo_so_erints.transpose(0,2,1,3)
cis_d_class = pyci.configint.CIS_D(ECIS, CCIS, eps_a, mo_so_erints, (nocc, nmo))

AttributeError: 'molecule' object has no attribute 'aoint'

In [ ]:
# MP2 energy calculation
print('\nComputing MP2 corrections\n')
mp2_e = psi4.energy('mp2')
print('PSI4 MP2 Ground state energy: %3.16f' % mp2_e)
print('CALC MP2 Ground state energy: %3.16f' % (scf_energy+cis_d_class.comp_e0mp2()))
print('Difference : %3.2E'%(scf_energy+cis_d_class.comp_e0mp2() - mp2_e))


In [ ]:
e_cis_d = cis_d_class.comp_cis_d(50, ncores=4)